# **Classificando Flores com KNN**

Iremos identificar tipos de flores do dataset *Iris* aplicando o algoritmo KNN. O dataset pode ser descarregado do link: https://www.kaggle.com/uciml/iris

In [1]:
# Carregando as bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
# Ignora os warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
# Carregando a base de dados "Iris" em um objeto do tipo DataFrame
dataset = pd.read_csv("Iris.csv")

In [3]:
# Visualizando os atributos
dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
# Removemos a coluna "Id" por ser irrelevante
dataset.drop('Id', axis = 1, inplace = True)

In [5]:
dataset.head(2)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [6]:
# Visualizando os valores únicos da classe
dataset['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

O dataset contém 5 atributos com informações da altura da sépala, largura da sépala, altura da pétala, largura da pétala, em cm, e a espécie de flor (atributo do tipo classe) com valores de: *Iris-setosa, Iris-versicolor e Iris-virginica*. A última coluna seria a rotulagem no Aprendizado Supervisionado. 

Estas informações serão passadas para o algoritmo, para que ele aprenda a classificar as espécies. Desta forma, posteriormente, quando alguém passar só as caracteristicas de uma determinada flor (sem a classificação) o algoritmo conseguiria identificar a que espécie pertence. Esta aprendizagem é feita com uma porção dos dados, tanto das caracteristicas quanto da classe.

Em um cenário de Aprendizado Não Supervisionado, também passaríamos os dados pro algoritmo, porém, sem as classes, só as características, pois aqui não teríamos uma rotulagem. Nesse caso, o algoritmo aprende a representar esses dados sem uma rotulagem (sem uma supervisão).

In [7]:
# Visualizando as informações do DataFrame
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


Temos 150 amostras, todas as features são do tipo float64, o atributo *Species* é do tipo object. Convertemos os tipos de dados float64 para float32 e object para category.

In [8]:
# Convertendo tipos de dados
dataset.SepalLengthCm = dataset.SepalLengthCm.astype('float32')
dataset.SepalWidthCm = dataset.SepalWidthCm.astype('float32')
dataset.PetalLengthCm = dataset.PetalLengthCm.astype('float32')
dataset.PetalWidthCm = dataset.PetalWidthCm.astype('float32')
dataset.Species = dataset.Species.astype('category')

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   SepalLengthCm  150 non-null    float32 
 1   SepalWidthCm   150 non-null    float32 
 2   PetalLengthCm  150 non-null    float32 
 3   PetalWidthCm   150 non-null    float32 
 4   Species        150 non-null    category
dtypes: category(1), float32(4)
memory usage: 2.7 KB


Com a conversão dos tipos de dados, obtivemos um ganho de memória.

In [10]:
# Estatística Descritiva
dataset.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843335,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


Os dados estatísticos nos permitem ter uma visão geral da base de dados.

**Dividindo os Dados em Treino e Teste**

O algoritmo KNN é um algoritmo do tipo Supervisionado, portanto, precisamos passar para ele dados das features e os rótulos, para ele aprender.

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.drop('Species', axis = 1), 
                                                    dataset['Species'], 
                                                    test_size = 0.3)

30% de nosso conjunto de dados será utilizado para Teste e 70% para Treino, a variável *X* contém as 4 features e a variável *y* a classe.

In [12]:
# Verificando a forma dos dados
X_train.shape, X_test.shape

((105, 4), (45, 4))

In [13]:
y_train.shape, y_test.shape

((105,), (45,))

**Instânciando o Algoritmo KNN**

A biblioteca `sklearn`, a través do pacote `neighbors`, contém o classificador do KNN. Importamos o classificador e o instanciamos:

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)

O parâmetro `n_neighbors` é o *k*, que define o número de vizinhos próximos. Executando `shift + tab` podemos lembrar os nomes dos atributos da classe `KNeighborsClassifier`.

**Treinando o Algoritmo**

Fazemos o treino do algoritmo KNN utilizando o método `fit`, um padrão da biblioteca `sklearn`, que recebe como parâmetros os dados de Treino.

In [15]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

**Executando o Algoritmo KNN com o Conjunto de Teste**

Utilizamos o método `predict`, também um padrão da biblioteca `sklearn`, que recebe como parâmetro apenas as features de Teste para fazer a predição.

In [16]:
result = knn.predict(X_test)
result

array(['Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-virginica', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-versicolor',
       'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-setosa', 'Iris-virginica', 'Iris-versicolor',
       'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-virginica',
       'Iris-virginica', 'Iris-virginica', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-setosa', 'Iris-virginica'], dtype=object)

A partir das features do conjunto de Teste, o algoritmo fez a predição de suas classes.

**Executando a Predição em Novas Amostras**

Passamos um array com valores dos 4 atributos que definem à espécie

In [17]:
data = np.array([[5.1, 3.5, 1.4, 0.2]])
knn.predict(data), knn.predict_proba(data)

(array(['Iris-setosa'], dtype=object), array([[1., 0., 0.]]))

O método `predict_proba` nos retorna a probabilidade que aquele dado de Teste pertença a uma respectiva classe. O método `predict`retorna um array com o valor predito da classificação.

Neste caso, a classificação foi *Iris-setosa*, a probabilidade do dado ser classificado como *Iris-setosa* é igual a 1, a probabilidade de pertencer às outras classes é de zero.

## **Técnicas de Validação e Verificação de Resultados**

*   **Métricas de Classificação**

O método `metrics.classification_report` recebe como parâmetros a classe de Teste, a predição do algoritmo para X_test, e os nomes das targets (nomes das classes).

In [18]:
from sklearn import metrics
print(metrics.classification_report(y_test, result, target_names = dataset['Species'].unique()))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        12
Iris-versicolor       1.00      0.88      0.94        17
 Iris-virginica       0.89      1.00      0.94        16

       accuracy                           0.96        45
      macro avg       0.96      0.96      0.96        45
   weighted avg       0.96      0.96      0.96        45



Temos as métricas de classificação, podemos observar que o valor médio de precisão foi de 96%, o algoritmo KNN acertou muito bem os dados.

*   **Matriz de Confusão**

Esta técnica permite ver como o algoritmo está se comportando em níveis de classes, saber em qual classe ele está errando mais. Aqui, executamos o método `crosstab` da biblioteca `Pandas`, que recebe como parâmetros a classe de Teste (y_test), a predição do algoritmo para X_test, o nome das linhas, o nome das colunas, e ativamos a opção para mostrar as margins.

In [19]:
print(pd.crosstab(y_test, result, rownames = ['Real'], colnames = ['                 Predito'], margins = True))

                 Predito  Iris-setosa  Iris-versicolor  Iris-virginica  All
Real                                                                       
Iris-setosa                        12                0               0   12
Iris-versicolor                     0               15               2   17
Iris-virginica                      0                0              16   16
All                                12               15              18   45


Para os valores reais *Iris-setosa*, o algoritmo classificou as 12 amostras como tais, não errou nenhuma. Para a classe *Iris-versicolor*, ele classificou 15 amostras como sendo *Iris-versicolor* e errou 2, que classificou como *Iris-virginica*. Para a classe *Iris-virginica* ele acertou as 16 amostras.